In [8]:
class Neuron(object):
    
    def __init__(self, set_of_input_weights, set_of_inputs, threshold):
        
        self.threshold = threshold
               
        self.set_of_input_weights = set_of_input_weights
        self.set_of_inputs = set_of_inputs
        self.num_of_inputs = len(self.set_of_inputs)   
        
    def set_threshold(self, value):
        self.threshold = value
    
    def output(self):
        length = self.num_of_inputs
        activation = 0
        output = 0
        for i in range(length):
            activation += self.set_of_input_weights[i] * self.set_of_inputs[i]
#             print "Activation: ", activation
        if activation > self.threshold:
            output = 1
        return output



In [9]:
weights = [0.5, 0.1, 0.4]
inputs = [1, 0, 1]
threshold = 0.5
n = Neuron(weights, inputs, 0.5)
# print n.output()


In [10]:
weights = {
    0: 0.01,
    1: 0.02,
    2: 0.1,
    3: 0.5,
    4: -0.6,
    5: 1,
    6: 1.5,
    7: 0.8,
    8: 0.05,
    9: -0.01,
    10: 0.005,
    11: 0.1,
    12: 0.5,
}
thresholds = {
    0: 0.05,
    1: 0.1,
    2: 0,
    3: 0.5,
    4: 1,
    5: 0.5,
}
inputs = [1, 0]
n0 = Neuron([weights[0], weights[3]], inputs, thresholds[0])
n1 = Neuron([weights[1], weights[2]], inputs, thresholds[1])
n2 = Neuron([weights[4], weights[7]], [n0.output(), n1.output()], thresholds[2])
n3 = Neuron([weights[5], weights[8]], [n0.output(), n1.output()], thresholds[3])
n4 = Neuron([weights[6], weights[9]], [n0.output(), n1.output()], thresholds[4])
n5 = Neuron([weights[10], weights[11], weights[12]], [n2.output(), n3.output(), n4.output()], thresholds[5])
# print n5.output()

In [26]:
import random as r
import copy


GOAL = 400

def init_pop(size):
    """
    in: desired size of population
    out: first random population
    """
    population = []
    for i in range(size):
        # from 0 to 144 1's
        population.append(r.randrange(0, 5708990770823839524233143877797980545530986495))
    return population


def fitness(chromosome):
    weights, thresholds = decode(chromosome)
    inputs = [[0, 0], [0, 1], [1, 0], [1, 1]]
    results = [0, 1, 1, 0]
    value = 0
    for i in range(4): # four posssible right answers
        if run_network(weights, thresholds, inputs[i]) == results[i]:
            value += 1
    return (10 - abs(value - 4))
    
def run_network(weights, thresholds, inputs):
    n0 = Neuron([weights[0], weights[3]], inputs, thresholds[0])
    n1 = Neuron([weights[1], weights[2]], inputs, thresholds[1])
    n2 = Neuron([weights[4], weights[7]], [n0.output(), n1.output()], thresholds[2])
    n3 = Neuron([weights[5], weights[8]], [n0.output(), n1.output()], thresholds[3])
    n4 = Neuron([weights[6], weights[9]], [n0.output(), n1.output()], thresholds[4])
    n5 = Neuron([weights[10], weights[11], weights[12]], [n2.output(), n3.output(), n4.output()], thresholds[5])
    return n5.output()

def is_solution(chromosome):
    if fitness(chromosome) == 10:
        return True
    return False

def select_partner(population):
    """
    in: population
    out: index of population selected for breeding
    method: roulette
    """
    fits = [fitness(x) for x in population]
    fit_sum = sum(fits)
    rnd = r.randrange(0, fit_sum)
    big_list = []
    for p in population:
        for f in xrange(fitness(p)):
            big_list.append(p)
    # print "big_list: ", big_list
    return big_list[rnd]


def crossover(parents):
    """
    in: parents [2]
    out: children [1]
    rate: 0.5
    """
    def get_first_half(c):
        return (c & 75557863725914323419135)
    def get_second_half(c):
        return (c & 5708990770823839524233068319934254631207567360)
    # crossover_decision = r.randrange(0, 1)
    crossover_decision = 1
    if crossover_decision:
        male_first_half = get_first_half(parents[0])
        female_second_half = get_second_half(parents[1])
        child = (male_first_half | female_second_half)
        return child
    else: return parents

def mutate(chromosome):
    """
    in: chromosome
    out: mutated chromosome
    rate: 1
    *
      note if rate is 0.5 takes significantly
      longer to find solution.
    *
    """
    bit_to_flip = r.randrange(0, 152)
    mutated_chrome = chromosome ^ (1 << bit_to_flip)
    return mutated_chrome

def average_fitness(population):
    """
    in: population
    out: sum of all fitness scores / length of population
    """
    fits = [fitness(x) for x in population]
    return (sum(fits)/len(fits))

def max_fitness(population):
    fits = [fitness(x) for x in population]
    return max(fits)

def chrome_to_string(chromosome):
    pass

def generations(gen_size):
    """
    pop = init_pop
    from 0 to generation do
        new_pop = []
        for p in pop
            if is_solution(p) do
                return p
            parents = [p, select_partner(pop)]
            children = [mutate(crossover(parents))]
            child = best(children)
            new_pop.append(child)
        pop = new_pop
    """
    population = init_pop(5)
    gen = 0
    for i in xrange(gen_size): # generations
        temp_population = []
        for chromosome in population:
            for c in decode(chromosome):
                print c
            if is_solution(chromosome):
                print "SOLUTION FOUND"
#                 print chrome_to_string(chromosome)
                return
            parents = [chromosome, select_partner(population)]
            crossed_child = crossover(parents)
            mutated_child = mutate(crossed_child)
            temp_population.append(mutated_child)
        gen += 1
        print "Generation: ", gen
        print "Maximum Fitness Score: ", max_fitness(population)
        print "Average Fitness Score: ", average_fitness(population)
        population = temp_population

generations(10)

def decode(chromosome):
    """
    in: bitstring
    out: decoded result
    """
    w0 = (chromosome & 255)
    w1 = ((chromosome & 65280) >> 8)
    w2 = ((chromosome & 16711680) >> 16)
    w3 = ((chromosome & 4278190080) >> 24)
    w4 = ((chromosome & 4278190080) >> 32)
    w5 = ((chromosome & 1095216660480) >> 40)
    w6 = ((chromosome & 280375465082880) >> 48)
    w7 = ((chromosome & 71776119061217280) >> 56)
    w8 = ((chromosome & 18374686479671623680) >> 64)
    w9 = ((chromosome & 4703919738795935662080) >> 72)
    w10 = ((chromosome & 1204203453131759529492480) >> 80)
    w11 = ((chromosome & 308276084001730439550074880) >> 88)
    w12 = ((chromosome & 78918677504442992524819169280) >> 96)
    t0 = ((chromosome & 20203181441137406086353707335680) >> 104)
    t1 = ((chromosome & 5172014448931175958106549077934080) >> 112)
    t2 = ((chromosome & 1324035698926381045275276563951124480) >> 120)
    t3 = ((chromosome & 338953138925153547590470800371487866880) >> 128)
    t4 = ((chromosome & 86772003564839308183160524895100893921280) >> 136)
    t5 = ((chromosome & 22213632912598862894889094373145828843847680) >> 144)
    weights = [w0, w1, w2, w3, w4, w5, w6, w7, w8, w9, w10, w11, w12]
    thresholds = [t0, t1, t2, t3, t4, t5]
    return weights, thresholds

[192L, 135L, 35L, 189L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[78L, 49L, 191L, 219L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[199L, 179L, 214L, 168L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[235L, 119L, 68L, 42L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[131L, 117L, 121L, 131L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
Generation:  1
Maximum Fitness Score:  8
Average Fitness Score:  8
[200L, 135L, 35L, 189L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[78L, 49L, 190L, 219L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[199L, 243L, 214L, 168L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[235L, 119L, 70L, 42L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
[131L, 116L, 121L, 131L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L, 0L]
[0L, 0L, 0L, 0L, 0L, 0L]
Generation:  2
Maximum Fitness Score:  8
Average Fitness Score:  8
[200L, 143L, 35L

In [25]:
x = ["0"]*76
print "".join(x)

0000000000000000000000000000000000000000000000000000000000000000000000000000
